In [ ]:
import subprocess

In [ ]:
subprocess.run('ls')

In [ ]:
import os, sys

# Open a file
path = "/Users/afaf/stage-afaf/images"
dirs = os.listdir( path )

# This would print all the files and directories
for file in dirs:
   print(file)

In [ ]:
dirs

In [ ]:
file_paths = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/images"):
  for filename in files:
    file_paths.append(os.path.abspath(os.path.join(folder, filename)))

file_paths

In [ ]:
for chemin_image in file_paths:
    myoquant= subprocess.run(['myoquant', "he-analysis", chemin_image,'--output-path',  '/Users/afaf/stage-afaf/data'])

    

In [ ]:
import pandas as pd

In [ ]:
df_30= pd.read_csv('data/30_results_summary.csv')
df_30

In [ ]:
df_30c=pd.read_csv('data/30_cell_details.csv')
df_30c

In [ ]:
fichier_details = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/data"):
  for filename in files:
    if filename.endswith(('_cell_details.csv')):
        fichier_details.append(os.path.abspath(os.path.join(folder, filename)))

fichier_details

In [ ]:
df=pd.read_csv('muscle_atlas_clean.csv') #atlas du muscle
df_cell=pd.read_csv('data/30_cell_details.csv')



def conversion_pix_µm(im, data_frame,muscle_atlas):
    'Convertion du diamètre en µm'
    barre_echelle= muscle_atlas.loc[(df['Number']== im), 'Calibration (um/px)']
    
    data_frame['diameter_µm']=(data_frame['feret_diameter_max'])*barre_echelle.values

conversion_pix_µm('5.jpg',df_cell,df )

def count_atrophic_cells (data_frame,min_size, max_size):
    hyper='Hypertrophy'
    hypo= 'Hypotrophy'
    norm= 'Normal'
    data_frame["State_cell"] = ''
    for index in data_frame.index:
        a=data_frame['diameter_µm'][index]
        if a > max_size:
            data_frame["State_cell"][index]=hyper
        elif a < min_size:
            data_frame["State_cell"][index]=hypo
        else: 
            data_frame["State_cell"][index]= norm


count_atrophic_cells (df_cell, 30, 80)

df_cell

In [ ]:
def pourcentage_state_cell (im,data_frame,muscle_atlas):
     data = {'Number':[],'Hypertrophy': [],'Normal':[],'Hypotrophy': []}  # Ici on créer un dataframe pour mettre ensuite les valeurs des pourcentages
     df_state_cell = pd.DataFrame(data)  
     percentage= (data_frame['State_cell'].value_counts()/data_frame['State_cell'].count())*100
     df_state_cell.loc[len(df_state_cell.index)] = percentage

     
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypertrophic_cell_%"]=df_state_cell['Hypertrophy'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Normal_cell_%"]=df_state_cell['Normal'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypotrophic_cell_%"]=df_state_cell['Hypotrophy'].values


pourcentage_state_cell ('30.jpg',df_cell,df)  
df.head(10)


In [ ]:
def diameter_mean_std (im,data_frame,muscle_atlas):
    cell_mean=data_frame['diameter_µm'].mean()
    cell_std=data_frame['diameter_µm'].std()
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_mean"]=cell_mean
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_std"]=cell_std


diameter_mean_std('30.jpg',df_cell,df)   
df.head(10)

In [ ]:
fichier = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/data"):
  for filename in files:
    if filename.endswith(('_results_summary.csv',)):
        fichier.append(os.path.abspath(os.path.join(folder, filename)))

fichier

In [ ]:
df_results_summary=pd.read_csv('data/30_results_summary.csv')
df3=df_results_summary.set_index('Feature')
df4=df3.transpose()
df4

In [ ]:
#test
df.loc[(df['Number']== '30.jpg'),"Proportion_nuclei"]=df4.loc['Raw Count','N° Nuclei']
df.head(10)

In [ ]:
def ajout_He (im,dataframe,muscle_atlas):
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_nuclei"]=dataframe.loc['Proportion (%)','N° Nuclei']
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_nuclei"]=dataframe.loc['Raw count','N° Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_nuclei"]=dataframe.iloc[0,0]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_intern_nuclei"]=dataframe.loc['Proportion (%)','N° Intern. Nuclei']
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_intern_nuclei"]=dataframe.loc['Raw Count','N° Intern. Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_intern_nuclei"]=dataframe.iloc[0,1]
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Periph_nuclei"]=dataframe.loc['Raw count','N° Periph. Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Periph_nuclei"]=dataframe.iloc[0,2]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_Periph_nuclei"]=dataframe.loc['Proportion (%)','N° Periph. Nuclei']
   # muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Cells with 1+ intern.nuc"]=dataframe.loc['Raw count','N° Cells with 1+ intern. nuc.']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Cells with 1+ intern.nuc"]=dataframe.iloc[0,3]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_Cells with 1+ intern. nuc"]=dataframe.loc['Proportion (%)','N° Cells with 1+ intern. nuc.']

    

ajout_He('30.jpg',df4,df)   
df.head(10)